# The Elastic Net

The elastic net is a penalized version of linear regression which assumes a model of the form
$$
\mathbf{Y} = \mathbf{X} \boldsymbol{\beta} + \boldsymbol{\varepsilon}
$$
where $\mathbf{Y}$ is a known $n \times 1$ observed response variable vector, $\mathbf{X}$ is a known $n \times (p + 1)$ observed explanatory variable matrix, $\boldsymbol{\beta}$ is an unknown $(p + 1) \times 1$ coefficient vector, and $\boldsymbol{\varepsilon}$ is an unknown vector of i.i.d. Gaussian errors with mean zero and unknown variance.
The classical least-squares estimator of $\boldsymbol{\beta}$ is
$$
\widehat{\boldsymbol{\beta}}_{\mathrm{LS}}
= \left(\mathbf{X}^\top \mathbf{X}\right)^- \mathbf{X}^\top \mathbf{Y},
$$
where $\left(\mathbf{X}^\top \mathbf{X}\right)^-$ is a generalized inverse of $\mathbf{X}^\top \mathbf{X}$.
Equivalently,
$$
\widehat{\boldsymbol{\beta}}_{\mathrm{LS}}
\in \mathrm{arg\,min}_{\boldsymbol{\beta} \in \mathbb{R}^{p + 1}}
\left(\frac{1}{2 n} \left\|\mathbf{Y} - \mathbf{X} \boldsymbol{\beta}\right\|_2^2\right).
$$
An *elastic net estimator* of $\boldsymbol{\beta}$ is any
$$
\widehat{\boldsymbol{\beta}}_{\mathrm{EN}}
\in \mathrm{arg\,min}_{\boldsymbol{\beta} \in \mathbb{R}^{p + 1}}
\left[\frac{1}{2 n} \left\|\mathbf{Y} - \mathbf{X} \boldsymbol{\beta}\right\|_2^2 + \lambda\left(\alpha \left\|\boldsymbol{\beta}\right\|_1 + \frac{1 - \alpha}{2} \left\|\boldsymbol{\beta}\right\|_2^2\right)\right]
$$
for some fixed $\alpha \in [0, 1]$ and $\lambda > 0$.

In [1]:
import numpy as np
from sklearn import datasets

from mltools.glm import LinearRegression, ElasticNet

In [2]:
# Set NumPy random number generator seed for replicability
np.random.seed(100)

## Load the Boston Housing Data

In [3]:
data = datasets.load_boston()

# Shuffle data
n = len(data.target)
idx = np.random.permutation(n)
x = data.data[idx, :]
y = data.target[idx]

# 50/50 training/testing split
x_train = x[:(n // 2), :]
x_test = x[(n // 2):, :]
y_train = y[:(n // 2)]
y_test = y[(n // 2):]

## Fit a Classical Linear Regression Model

In [4]:
%%time
model = LinearRegression().fit(x_train, y_train)

CPU times: user 2.26 ms, sys: 1.72 ms, total: 3.98 ms
Wall time: 5.54 ms


### Report Mean Squared Error

In [5]:
mse_train = model.mse(x_train, y_train)
mse_test = model.mse(x_test, y_test)
print(f"Training MSE: {mse_train:.3f}")
print(f"Testing MSE:  {mse_test:.3f}")

Training MSE: 23.630
Testing MSE:  23.131


## Fit an Elastic Net Model

In [6]:
%%time
model = ElasticNet(alpha=0.5, lam=0.01).fit(x_train, y_train)

CPU times: user 34.2 ms, sys: 2.02 ms, total: 36.2 ms
Wall time: 39 ms


### Report Mean Squared Error

In [7]:
mse_train = model.mse(x_train, y_train)
mse_test = model.mse(x_test, y_test)
print(f"Training MSE: {mse_train:.3f}")
print(f"Testing MSE:  {mse_test:.3f}")

Training MSE: 23.843
Testing MSE:  22.271


## References

* Zou Hui, Trevor Hastie.
  "Regularization and Variable Selection via the Elastic Net".
  Journal of the Royal Statistical Society, Series B. Vol. 67, No. 2, 2005, pp. 301--320.
  ([DOI](https://doi.org/10.1111/j.1467-9868.2005.00503.x))
* Jerome Friedman, Trevor Hastie, and Robert Tibshirani.
  "Regularization Paths for Generalized Linear Models via Coordinate Descent".
  Journal of Statistical Software Vol. 33, No. 1, 2010, pp. 1--22.
  ([PMCID](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2929880/))